In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline

In [61]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    random_state=42)

In [62]:
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [63]:
svm = SVC()
svm.fit(X_train_scaled, y_train)
X_test_scaled = scaler.transform(X_test)

In [64]:
svm.score(X_test_scaled, y_test)

0.9790209790209791

## 최적화

In [65]:
param_grid = {"C":[0.001,0.01,0.1,1,10,100],
              "gamma":[0.001,0.01,0.1,1,10,100]}
grid = GridSearchCV(SVC(),param_grid=param_grid)
grid.fit(X_train_scaled, y_train)
print(grid.best_score_)
grid.score(X_test_scaled,y_test)
grid.best_params_

0.9788508891928865


{'C': 1, 'gamma': 1}

# 파이프라인 구축

In [66]:
pipe = Pipeline([("scaler", MinMaxScaler()),("svm",SVC())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9790209790209791

In [67]:
param_grid = {"svm__C":[0.001,0.01,0.1,1,10,100],
              "svm__gamma":[0.001,0.01,0.1,1,10,100]}
grid = GridSearchCV(pipe,param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
grid.score(X_test_scaled,y_test)
print(grid.best_score_,grid.best_params_)

0.9741450068399453 {'svm__C': 10, 'svm__gamma': 0.1}


## 파이프라인 인터페이스

In [68]:
pipe_short = make_pipeline(MinMaxScaler(),SVC(C=100))
pipe_short

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))])

In [69]:
pipe_short.steps

[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))]

In [70]:
pipe_short = make_pipeline(StandardScaler(),PCA(n_components=2),StandardScaler())
pipe_short.fit(cancer.data)

Pipeline(steps=[('standardscaler-1', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('standardscaler-2', StandardScaler())])

In [71]:
components = pipe_short.named_steps["pca"].components_
print(components)

[[ 0.21890244  0.10372458  0.22753729  0.22099499  0.14258969  0.23928535
   0.25840048  0.26085376  0.13816696  0.06436335  0.20597878  0.01742803
   0.21132592  0.20286964  0.01453145  0.17039345  0.15358979  0.1834174
   0.04249842  0.10256832  0.22799663  0.10446933  0.23663968  0.22487053
   0.12795256  0.21009588  0.22876753  0.25088597  0.12290456  0.13178394]
 [-0.23385713 -0.05970609 -0.21518136 -0.23107671  0.18611302  0.15189161
   0.06016536 -0.0347675   0.19034877  0.36657547 -0.10555215  0.08997968
  -0.08945723 -0.15229263  0.20443045  0.2327159   0.19720728  0.13032156
   0.183848    0.28009203 -0.21986638 -0.0454673  -0.19987843 -0.21935186
   0.17230435  0.14359317  0.09796411 -0.00825724  0.14188335  0.27533947]]


## 간단 예제(교재 p.430)

In [72]:
# 가능하면 전처리를 엑셀 혹은 구글 스프레드시트등을 활용하기
# vscode에서 텍스트 전처리를 진행하기.

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url,sep=r"\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2,:],raw_df.values[1::2,:2]])
target = raw_df.values[1::2,2]
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    random_state=42)

In [73]:
pipe = make_pipeline(StandardScaler(), PolynomialFeatures(), Ridge())

In [74]:
param_grid = {"polynomialfeatures__degree":[1,2,3],
              "ridge__alpha" : [0.001,0.01,0.1,1,10,100]}

In [75]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('ridge', Ridge())]),
             n_jobs=-1,
             param_grid={'polynomialfeatures__degree': [1, 2, 3],
                         'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]})

In [76]:
grid.score(X_test,y_test)

0.8054402042295686

## 모델 선택을 위한 파이프라인

In [88]:
pipe = Pipeline([("preprocessing",StandardScaler()),('classifier',SVC())], memory="cache_folder")

In [89]:
param_grid = [
    {"classifier":[SVC()],
     "preprocessing":[StandardScaler()],
     "classifier__C":[0.001,0.01,0.1,1,10,100]},
    {"classifier":[RandomForestClassifier()],
     "preprocessing":[None],
     "classifier__max_features":[1,2,3]}
]

In [90]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    random_state=42)

In [91]:
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(memory='cache_folder',
                                steps=[('preprocessing', StandardScaler()),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'preprocessing': [StandardScaler()]},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__max_features': [1, 2, 3],
                          'preprocessing': [None]}])

In [92]:
grid.best_params_

{'classifier': SVC(), 'classifier__C': 1, 'preprocessing': StandardScaler()}

In [93]:
grid.best_score_

0.9717920656634748

In [94]:
grid.score(X_test,y_test)

0.972027972027972

데이터처리 
감정분석 => 자연어처리 => 문자열처리 > 원핫인코딩 X 임베딩!!! / 다른 외국어와 다르게 한국어는 별도처리! / 설치가 엄청 빡셈,,